In [ ]:
import pandas as pd
import signatureanalyzer as sa

# File paths and variable constants (REPLACE INPUTS)
REF = "cosmic3"
REF_PATH = "sa_cosmic3_sbs_exome.tsv"
MAF_PATH = "example.maf"
HG_PATH = "hg38.2bit"
SIG_LIST = ['SBS1','SBS2','SBS3']
MAX_ITER = 30000

# Extract mutational spectra
maf = pd.read_csv(MAF_PATH,sep='\t')
spectra_df = sa.spectra.get_spectra_from_maf(maf, hgfile=HG_PATH, reference=REF)[1]

# Import signature reference data
ref_df = pd.read_csv(REF_PATH, sep='\t',index_col=0)

# Map spectra index to ref index
spectra_df.index = sa.utils._map_sbs_id_sigs(spectra_df,ref_df,REF)

# Subset reference data to create W matrix
Wref_df = ref_df.set_index('Somatic Mutation Type').iloc[:,:-2]
Wref_df = Wref_df.loc[:, SIG_LIST]

# Run supervised NMF
res_supervised = sa.supervised_bnmf.supervised_ardnmf(
    spectra_df,
    Wref_df,
    objective='poisson',
    verbose=True,
    max_iter=MAX_ITER)

# Visualized Stacked Barplot for Attributions
_ = sa.pl.stacked_bar(res_supervised['H'], REF)
